In [2]:
%%writefile transform.py

import numpy as np
from dataclasses import dataclass

import constants as C

@dataclass
class BLH:
    lat : np.array
    lng : np.array
    hgt : np.array

def haversine_distance(blh_1, blh_2):
    dlat = blh_2.lat - blh_1.lat
    dlng = blh_2.lng - blh_1.lng
    a = np.sin(dlat/2)**2 + np.cos(blh_1.lat) * np.cos(blh_2.lat) * np.sin(dlng/2)**2
    dist = 2 * C.HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist

def pd_haversine_distance(df1, df2):
    blh1 = BLH(
        lat=np.deg2rad(df1['latDeg'].values),
        lng=np.deg2rad(df1['lngDeg'].values),
        hgt=0,
    )
    blh2 = BLH(
        lat=np.deg2rad(df2['latDeg'].values),
        lng=np.deg2rad(df2['lngDeg'].values),
        hgt=0,
    )
    return haversine_distance(blh1, blh2)

Overwriting transform.py


In [3]:

%%writefile area_prediction.py

import numpy as np
import pandas as pd
from pathlib import Path
from glob import glob
from sklearn.neighbors import KNeighborsClassifier

BASE_DIR = Path('./input/google/google-smartphone-decimeter-challenge')

train_base = pd.read_csv(BASE_DIR / 'baseline_locations_train.csv')
train_base = train_base.sort_values([
    "collectionName", "phoneName", "millisSinceGpsEpoch"
]).reset_index(drop=True)

train_base['area'] = train_base['collectionName'].map(lambda x: x.split('-')[4])

train_name = np.array(sorted(path.split('/')[-1] for path in glob(f'{BASE_DIR}/train/*')))
train_highway  = train_name[np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]) - 1]
train_tree     = train_name[np.array([22,23,25,26,28]) - 1]
train_downtown = train_name[np.array([24,27,29]) - 1]

train_base['area_target'] = -1
train_base.loc[train_base['collectionName'].isin(train_highway),  'area_target'] = 0
train_base.loc[train_base['collectionName'].isin(train_tree),     'area_target'] = 1
train_base.loc[train_base['collectionName'].isin(train_downtown), 'area_target'] = 2

def processing_downtown(input_df: pd.DataFrame, is_train=False):
    output_df = input_df.groupby('collectionName')[['latDeg', 'lngDeg']].std()
    if is_train:
        output_df = output_df.merge(
            input_df.groupby('collectionName')[['area_target']].first(),
            on='collectionName')
    output_df = output_df.merge(
        input_df.groupby('collectionName')['area'].first(),
        on='collectionName')
    output_df = output_df.merge(
        input_df.groupby('collectionName')['phoneName'].unique().apply(list),
        on='collectionName')
    return output_df

train = processing_downtown(train_base, is_train=True)
train['downtown_target'] = (train['area_target']==2).astype(int)

downtown_model_knn = KNeighborsClassifier(n_neighbors=1)
downtown_model_knn.fit(
    train[['latDeg', 'lngDeg']],
    train['downtown_target'],
)

def processing_highway_tree(input_df: pd.DataFrame, is_train=False):
    output_df = input_df.groupby('collectionName')[['latDeg', 'lngDeg']].min()
    if is_train:
        output_df = output_df.merge(
            input_df.groupby('collectionName')[['area_target']].first(),
            on='collectionName')
    output_df = output_df.merge(
        input_df.groupby('collectionName')['area'].first(),
        on='collectionName')
    output_df = output_df.merge(
        input_df.groupby('collectionName')['phoneName'].unique().apply(list),
        on='collectionName')
    return output_df

train = processing_highway_tree(train_base, is_train=True)

highway_tree_model_knn = KNeighborsClassifier(n_neighbors=1)
highway_tree_model_knn.fit(
    train.loc[train['area_target']!=2, ['latDeg', 'lngDeg']],
    train.loc[train['area_target']!=2, 'area_target'],
)

def predict_area(test_base):
    test_base = test_base.copy()
    test_base = test_base.sort_values([
        "collectionName", "phoneName", "millisSinceGpsEpoch"
    ]).reset_index(drop=True)
    test_base['area'] = test_base['collectionName'].map(lambda x: x.split('-')[4])

    test = processing_downtown(test_base)
    downtown_pred = downtown_model_knn.predict(test[['latDeg', 'lngDeg']])

    test = processing_highway_tree(test_base)
    test.loc[downtown_pred==1, 'area_pred'] = 2
    pred = highway_tree_model_knn.predict(test.loc[test['area_pred'].isnull(), ['latDeg', 'lngDeg']])
    test.loc[test['area_pred'].isnull(), 'area_pred'] = pred
    test['area_pred'] = test['area_pred'].astype(int)
    test['collectionName'] = test.index

    test_highway  = []
    test_tree     = []
    test_downtown = []
    for collection, area_pred in test[['collectionName', 'area_pred']].itertuples(index=False):
        if area_pred == 0:
            test_highway.append(collection)
        elif area_pred == 1:
            test_tree.append(collection)
        else:
            test_downtown.append(collection)
    return (test_highway, test_tree, test_downtown)

Overwriting area_prediction.py


In [4]:
import multiprocessing
import glob
import numpy as np
import pandas as pd
import scipy.sparse
import scipy.sparse.linalg
from tqdm.notebook import tqdm

import transform
import area_prediction

INPUT_PATH = './input/google/google-smartphone-decimeter-challenge'

BASELINE_DF = pd.concat([pd.read_csv(f'{INPUT_PATH}/baseline_locations_train.csv'),
                         pd.read_csv(f'{INPUT_PATH}/baseline_locations_test.csv'),
                         ], axis=0)
def get_baseline(collection_name):
    df = BASELINE_DF[BASELINE_DF['collectionName'] == collection_name].copy()
    df.reset_index(drop=True, inplace=True)
    return df

def get_optimization_constants(base_df, sigma_y):
    const = dict()
    dt = 1.0
    t0 = base_df['millisSinceGpsEpoch'].min()
    TIME_y = 1e-3 * (base_df['millisSinceGpsEpoch'] - t0).values
    N_y = TIME_y.shape[0]
    N_x = int(np.ceil(np.max(TIME_y) / dt) + 1)
    const['N_y'] = N_y
    const['N_x'] = N_x

    a = np.array([[1, dt, (1/2)*dt**2],
                  [0,  1,  dt],
                  [0,  0,  1]])
    e3 = scipy.sparse.eye(3)
    A = np.empty(shape=(2*(N_x-1), 2*N_x), dtype=np.object)
    for i_x in range(N_x-1):
        A[2*i_x  , 2*i_x  ] = a
        A[2*i_x+1, 2*i_x+1] = a
        A[2*i_x  , 2*i_x+2] = -e3
        A[2*i_x+1, 2*i_x+3] = -e3
    const['A'] = scipy.sparse.bmat(A, format='csr')
    
    b = np.array([[(1/6)*dt**3,
                   (1/2)*dt**2,
                   dt]]).T
    const['B'] = scipy.sparse.block_diag([b for _ in range(2*(N_x-1))], format='csr')

    sigma_u = 1.0
    diag_R  = np.full(2*N_x - 2, sigma_u**(-2) * dt)
    const['R'] = scipy.sparse.spdiags(diag_R, [0], 2*N_x - 2, 2*N_x - 2, format='csc')
    
    x_index  = np.floor(TIME_y / dt).astype(int)
    alpha    = (TIME_y / dt) - x_index
    coeff_y0 = 1 - 3*alpha**2 + 2*alpha**3
    coeff_y1 =     3*alpha**2 - 2*alpha**3
    coeff_v0 = alpha * (alpha - 1)**2
    coeff_v1 = alpha**2 * (alpha - 1)
    C = np.empty(shape=(2*N_y, 2*N_x), dtype=np.object)
    for i_x in range(N_x):
        C[0, 2*i_x  ] = scipy.sparse.coo_matrix((1, 3))
        C[0, 2*i_x+1] = scipy.sparse.coo_matrix((1, 3))
    for i_y in range(N_y):
        i_x = x_index[i_y]
        c_i = np.array([[coeff_y0[i_y], coeff_v0[i_y], 0]])
        C[2*i_y,   2*i_x]   = c_i
        C[2*i_y+1, 2*i_x+1] = c_i
        if i_x < N_x - 1:
            c_iplus = np.array([[coeff_y1[i_y], coeff_v1[i_y], 0]])
            C[2*i_y,   2*i_x+2] = c_iplus
            C[2*i_y+1, 2*i_x+3] = c_iplus
    const['C_orig']  = scipy.sparse.bmat(C, format='csr')

    diag_L = np.full(2*N_y, sigma_y**(-2))
    const['L_orig'] = scipy.sparse.spdiags(diag_L, [0], 2*N_y, 2*N_y, format='csr')

    const['Y_orig'] = base_df[['latDeg', 'lngDeg']].values.flatten()
    
    return const

def solve_QP(const, valid):
    A = const['A']
    B = const['B']
    R = const['R']
    C_orig = const['C_orig']
    L_orig = const['L_orig']
    Y_orig = const['Y_orig']
    valid2 = np.stack([valid, valid], axis=1).flatten()
    C = C_orig[valid2, :]
    L = L_orig[np.ix_(valid2, valid2)]
    Y = Y_orig[valid2]
    
    BRB = B @ scipy.sparse.linalg.spsolve(R, B.T)
    CLC = C.T @ (L @ C)
    CLY = C.T @ (L @ Y)
    A_sys  = scipy.sparse.bmat([[CLC, A.T], [A, -BRB]], format='csc')
    b_sys  = np.concatenate([CLY, np.zeros(A.shape[0])], axis=0)
    x_sys  = scipy.sparse.linalg.spsolve(A_sys, b_sys)
    X_star = x_sys[0:A.shape[1]]
    Y_star = C_orig @ X_star
    return Y_star

def do_postprocess(args):
    collection_name, params = args
    base_df = get_baseline(collection_name)
    const   = get_optimization_constants(base_df, params['sigma_y'])
    valid   = np.full(const['N_y'], True)
    for loop in range(3):
        Y_star = solve_QP(const, valid)
        Y_star = np.reshape(Y_star, (-1, 2))
        pp_df  = base_df.copy()
        pp_df['latDeg'] = Y_star[:, 0]
        pp_df['lngDeg'] = Y_star[:, 1]
        d = transform.pd_haversine_distance(pp_df, base_df)
        valid = (d < params['reject_m'])
    return pp_df

def make_postprocessing_df(config):
    args_list = []
    for collection_list, params in config:
        for collection_name in collection_list:
            args_list.append((collection_name, params))
    processes = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=processes) as pool:
        df_list = pool.imap_unordered(do_postprocess, args_list)
        df_list = tqdm(df_list, total=len(args_list))
        df_list = list(df_list)
    output_df = pd.concat(df_list, axis=0).sort_values(['phone', 'millisSinceGpsEpoch'])
    return output_df

def print_score(output_df):
    score_list = []
    for gid, phone_df in output_df.groupby('phone'):
        drive, phone = gid.split('_')
        gt_df = pd.read_csv(f'{INPUT_PATH}/train/{drive}/{phone}/ground_truth.csv')
        d = transform.pd_haversine_distance(phone_df, gt_df)
        score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])
        score_list.append(score)
    score = np.mean(score_list)
    print(f'train score: {score:.3f}')
    return

def main():
    params_highway = { 'sigma_y'  : 3.0,
                       'reject_m' : 7.0,
                      }
    params_treeway = { 'sigma_y'  : 6.0,
                       'reject_m' : 11.0,
                      }
    params_downtown = { 'sigma_y'  : 30.0,
                        'reject_m' : 19.0,
                       }

    collection_list_all = np.array(sorted(path.split('/')[-1] for path in glob.glob(f'{INPUT_PATH}/train/*')))
    collection_list_highway  = collection_list_all[np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]) - 1]
    collection_list_treeway  = collection_list_all[np.array([22,23,25,26,28]) - 1]
    collection_list_downtown = collection_list_all[np.array([24,27,29]) - 1]
    config = [
        (collection_list_highway,  params_highway),
        (collection_list_treeway,  params_treeway),
        (collection_list_downtown, params_downtown),
    ]
    train_pp_df = make_postprocessing_df(config)
    print_score(train_pp_df)
    
    test_base = pd.read_csv(f'{INPUT_PATH}/baseline_locations_test.csv')
    collection_list_highway, collection_list_treeway, collection_list_downtown = area_prediction.predict_area(test_base)
    config = [
        (collection_list_highway,  params_highway),
        (collection_list_treeway,  params_treeway),
        (collection_list_downtown, params_downtown),
    ]
    test_pp_df = make_postprocessing_df(config)

    train_pp_df.to_csv('smoothing_1st_train.csv', index=False)
    test_pp_df.to_csv('smoothing_1st_test.csv', index=False)

    columns = ['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']
    sub_df = test_pp_df[columns]
    sub_df.to_csv('submission10.csv', index=False)
    return

In [5]:
main()

  0%|          | 0/29 [00:00<?, ?it/s]

/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object`

train score: 3.309


  0%|          | 0/19 [00:00<?, ?it/s]

/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/centos/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object`